In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

# def printmem():
#     allocated = torch.cuda.memory_allocated()
#     cached = torch.cuda.memory_cached()
#     print('Allocated:', str(allocated), '['+str(round(allocated/1000000000,3))+' GB]')
#     print('   Cached:', str(cached), '['+str(round(cached/1000000000,3))+' GB]')

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
    
# printmem()



In [2]:
from criteria import KIDMindLoss
from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import KID_Data
from modules import Mind_of_KID
import utils

# printmem()

In [3]:
# Load the data
main_dir = '/Volumes/ALEX USB/project_KID/'
train_dir = main_dir + 'KID_memory/'
# test_dir = main_dir + 'low_AD/'
modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

train_ds = KID_Data(folder=train_dir, crop=55, scale=1, stride=1)
# test_ds = MIBIData(folder=test_dir, crop=32, scale=10, stride=16)

finished loading
There are  10000 samples


In [4]:
laddernet_args = dict()

# laddernet_args['batchnorm'] = False

kidmind = Mind_of_KID()
# kidmind = Mind_of_KID.load_model('/Volumes/ALEX USB/project_KID/models/2019Aug17_15-33-34/', 'model_777')
# laddernet.cuda()
# print(owlnet)

In [5]:
kidmind_logger = Logger(['loss'])

In [6]:
kidmind_trainer = Trainer()

In [7]:
# LadderNet training parameters
kidmind_train_args = dict()
kidmind_train_args['lr'] = 0.01
kidmind_train_args['batch_size'] = 100
kidmind_train_args['epochs'] = 5
kidmind_train_args['report'] = 5
kidmind_train_args['crop'] = 55
kidmind_train_args['clip'] = None
kidmind_train_args['decay'] = 0
kidmind_train_args['restart'] = False
kidmind_train_args['epoch_frac'] = 1
# laddernet_train_args['decay'] = 1e-5

# LadderNet loss parameters
kidmind_loss_args = {
    'alpha': 1,
    'beta': 1,
    'gamma': 0,
    'delta': 0.1
}

train_ds.set_crop(kidmind_train_args['crop'])

In [ ]:
kidmind_train_args['continue'] = False
# laddernet.set_noise_std(0)
# laddernet.set_num_steps(10)
kidmind_criterion = KIDMindLoss(**kidmind_loss_args)
kidmind_trainer.train(kidmind, train_ds, kidmind_criterion, kidmind_logger, '/Volumes/ALEX USB/project_KID/models/', **kidmind_train_args)
# print()
# torch.cuda.empty_cache()
# printmem()

    Minibatch:58/99 > < loss: 0.92647, inv_loss: 0.87401, for_loss: 0.21361, sen_loss: 0.05203, per_loss: 0.00431,                                                                                                       

In [ ]:
kidmind.eval()
batch = train_ds.get_samples([np.random.randint(10000)])
imgx = batch['s_i'][0,:,:,:].transpose(0,1).transpose(1,2)
y = kidmind.forward(**batch)
imgy = y['~s_i'][0,:,:,:].transpose(0,1).transpose(1,2).detach()

fig = plt.figure(figsize=[10,5])
ax1 = plt.subplot(1,2,1)
plt.imshow(imgx)
ax2 = plt.subplot(1,2,2)
plt.imshow(imgy)

In [ ]:
import numpy as np
np.random.randint(10)

In [ ]:
np.random.permutation(9)+1